<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/fastai_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from IPython.display import display,HTML

In [ ]:
from fastai.text.all import *

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders = ['train','test', 'unsup'])
files

(#100000) [Path('/root/.fastai/data/imdb/test/neg/2270_1.txt'),Path('/root/.fastai/data/imdb/test/neg/3302_1.txt'),Path('/root/.fastai/data/imdb/test/neg/3869_1.txt'),Path('/root/.fastai/data/imdb/test/neg/12406_1.txt'),Path('/root/.fastai/data/imdb/test/neg/6582_1.txt'),Path('/root/.fastai/data/imdb/test/neg/11519_3.txt'),Path('/root/.fastai/data/imdb/test/neg/6496_4.txt'),Path('/root/.fastai/data/imdb/test/neg/2903_1.txt'),Path('/root/.fastai/data/imdb/test/neg/10505_1.txt'),Path('/root/.fastai/data/imdb/test/neg/12135_3.txt')...]

In [ ]:
txt = files[0].open().read()
txt[:75]

'The comparisons between the 1995 version and this are inevitable. Sadly, th'

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
toks

(#636) ['The','comparisons','between','the','1995','version','and','this','are','inevitable'...]

In [ ]:
print(coll_repr(toks,30))

(#636) ['The','comparisons','between','the','1995','version','and','this','are','inevitable','.','Sadly',',','this','version','falls','far','short.<br','/><br','/>The','casting','is','uninspired','and','the','acting','wooden','.','One','gets'...]


In [ ]:
first(spacy(['The U.S.dollar $1 is $1.00']))

(#7) ['The','U.S.dollar','$','1','is','$','1.00']

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

# xxbos:: Indicates the beginning of a text (here, a review)
# xxmaj:: Indicates the next word begins with a capital (since we lowercased everything)
# xxunk:: Indicates the word is unknown

(#729) ['xxbos','xxmaj','the','comparisons','between','the','1995','version','and','this','are','inevitable','.','xxmaj','sadly',',','this','version','falls','far','short','.','\n\n','xxmaj','the','casting','is','uninspired','and','the','acting'...]


In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [ ]:
coll_repr(tkn('&copy; Fart.ai www.fast.ai/INDEX'),31)

"(#11) ['xxbos','©','xxmaj','fart.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [ ]:
txts = L(o.open().read() for o in files[:2000])
txts

(#2000) ['The comparisons between the 1995 version and this are inevitable. Sadly, this version falls far short.<br /><br />The casting is uninspired and the acting wooden. One gets the impression the director did not read the book,so did not understand the characters.<br /><br />Sir Walter Elliot is portrayed as pompous but his inadvertent silliness which Redgrave brilliantly captured (in the 1995 version) is nowhere to be found.<br /><br />The Musgrove sisters are so unlikable, one doesn\'t understand why Wentworth or anybody else would give them a second glance.<br /><br />The relationship between Wentworth and Anne is devoid of feeling.<br /><br />In the 1995 version, Hinds and Root managed to convey the depth of emotion the two of them felt towards each other with their body language and facial expressions. In this one, it is hard to understand Anne spent years mired in regret unless one has read the book.<br /><br />This production does not capture the emotional complexity of the

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz = sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
# subword(1000)

In [ ]:
# subword(200)

In [ ]:
# subword(10000)

In [ ]:
print(coll_repr(tkn(txt),31))

(#729) ['xxbos','xxmaj','the','comparisons','between','the','1995','version','and','this','are','inevitable','.','xxmaj','sadly',',','this','version','falls','far','short','.','\n\n','xxmaj','the','casting','is','uninspired','and','the','acting'...]


In [ ]:
toks = tkn(txt)
toks

(#729) ['xxbos','xxmaj','the','comparisons','between','the','1995','version','and','this'...]

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#729) ['xxbos','xxmaj','the','comparisons','between','the','1995','version','and','this'...]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

'(#2048) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'the\',\',\',\'.\',\'a\',\'and\',\'to\',\'of\',\'is\',\'in\',\'it\',\'"\'...]'

In [ ]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,    9, 1510,  204,    9,  550,  268,   13,   21,   36, 1511,   11,    8,  551,   10,   21,  268,    0,  189])

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj the comparisons between the 1995 version and this are inevitable . xxmaj sadly , this version xxunk far'

In [ ]:
stream ="In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs, seq_len = 6, 15
d_tokens = np.array([tokens[i*seq_len:(i+1)* seq_len] for i in range(bs)])
d_tokens
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index = False, header = None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
bs , seq_len = 6, 5
d_tokens = np.array([tokens[i*15:i*15 + seq_len] for i in range(bs)])
df= pd.DataFrame(d_tokens)
display(HTML(df.to_html(index = False, header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [ ]:
bs , seq_len = 6, 5
d_tokens = np.array([tokens[i*15+ seq_len:i*15 + 2*seq_len] for i in range(bs)])
df= pd.DataFrame(d_tokens)
display(HTML(df.to_html(index = False, header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [ ]:
bs, seq_len = 6,5
d_tokens = np.array([tokens[i*15 + 10 : i * 15 + 15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False, header = None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [ ]:
nums200 = toks200.map(num) 

In [ ]:
# nums200

In [ ]:
dl = LMDataLoader(nums200) #データ整形（1つ前のデータから1つ単語をシフトさせてデータを作る）

In [ ]:
x, y = first(dl)# 最初のbatchを取得
x.shape, y.shape #(batchサイズ, トークン数) 

(torch.Size([64, 72]), torch.Size([64, 72]))

###*   例:x.shape(1, 7), y.shape(1, 7)

*   x(input)「 昨日　の　夜　何　食べ　た　？」(1,7)
*   ↓
*   y(output)「の　夜　何　食べ　た　？　**カレー**」(1,7)　






In [ ]:
get_imdb = partial(get_text_files, folders = ['train', ' test', 'unsup'])

dls_lm = DataBlock(
    #ディレクトリからデータを読み込む、is_lm=TrueでLMDataLoadeと同じことをやっている
    blocks = TextBlock.from_folder(path, is_lm=True),
    #pathからテキストを読み込む
    get_items = get_imdb,splitter=RandomSplitter(0.1)
    #90%で学習、10%で評価
    ).dataloaders(path, path=path,bs=32, seq_len=80)


In [ ]:
dls_lm.show_batch(max_n = 2)

,text,text_
0,"xxbos xxmaj man … i thought xxmaj i 'd seen some bad movies before , but this is seriously the worst ever . xxmaj its so bad , that if you are even the least bit drunk or high you 'll find this movie hilarious . xxmaj there are 6 - 8 totally ugly chicks who all just keep getting naked . xxmaj the production value is zero , think high school project done in a weekend . xxmaj it","xxmaj man … i thought xxmaj i 'd seen some bad movies before , but this is seriously the worst ever . xxmaj its so bad , that if you are even the least bit drunk or high you 'll find this movie hilarious . xxmaj there are 6 - 8 totally ugly chicks who all just keep getting naked . xxmaj the production value is zero , think high school project done in a weekend . xxmaj it was"
1,". xxmaj respect ! xxbos xxmaj three likable and spirited actresses are stuck with weak material here . xxmaj the film sags in the first hour because there is too much exposition ( and some attempts at social - drama undertones ) , and it does n't much improve when the actual heist begins . xxmaj there is not much fun to be had here . xxmaj do n't bother watching this . ( * * ) xxbos i think","xxmaj respect ! xxbos xxmaj three likable and spirited actresses are stuck with weak material here . xxmaj the film sags in the first hour because there is too much exposition ( and some attempts at social - drama undertones ) , and it does n't much improve when the actual heist begins . xxmaj there is not much fun to be had here . xxmaj do n't bother watching this . ( * * ) xxbos i think it"


In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
# learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
# learn.fit_one_cycle(1,2e-3)

In [ ]:
learn.save_encoder('finetuned')#エンコーダのみ保存

In [ ]:
TEXT = 'I liked this movie because'
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS,temperature=0.75)
                for _ in range(N_SENTENCES)]


In [ ]:
print('/n'.join(preds))

i liked this movie because of its " very strong , wonderfully realistic scenes . " In an interview , Martin stated that the film was " about getting better " and that it was intended for the United States/ni liked this movie because of its geographical location , but it was also a fun extension of the British Indian Army . The film , according to Sam Simon , it was " from the Cinema


In [ ]:
dls_clas = DataBlock(
    #データはテキスト(TextBlock)、正解はカテゴリ(CategoryBlock)、vocabは前回と同じ(dls_lm)
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    #1つ上の階層からポジorネガを持って来る(/neg/)
    get_y = parent_label,
    #folders=['train','test']からデータを読み込む
    get_items=partial(get_text_files, folders=['train', 'test']),
    #'test'デレクとり以下を評価用
    splitter = GrandparentSplitter(valid_name='test')
    #batchサイズ32、単語の処理数72
    ).dataloaders(path, path=path, bs=32, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxrep 3 * xxup spoilers xxrep 3 * xxrep 3 * xxup spoilers xxrep 3 * xxmaj continued … \n\n xxmaj from here on in the whole movie collapses in on itself . xxmaj first we meet a rogue program with the indication we 're gon na get ghosts and vampires and werewolves and the like . xxmaj we get a guy with a retarded accent talking endless garbage , two ' ghosts ' that serve no real purpose and have no character what - so - ever and a bunch of henchmen . xxmaj someone 's told me they 're vampires ( straight out of xxmaj blade 2 ) , but they 're so undefined i did n't realise . \n\n xxmaj the funny accented guy with a ridiculous name suffers the same problem as the xxmaj oracle , only for far longer and far far worse .",neg


In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')#最後のレイヤーのみを学習(load)

In [ ]:
# leran.fit_one_cycle(1,2e-2)

In [ ]:
#最後から2番目までのレイヤーを解凍し、学習対象とする、それより前は固定
# learn.freeze_to(-2) 
# learn.fit_one_cycle(1,slice(1e-2 / (2.6**4),1e-2))

In [ ]:
#最後から3番目までのレイヤーを解凍し、学習対象とし、それより前は固定
# learn.freeze_to(-3)
# learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
#すべて学習対象
# learn.unfreeze()
# learn.fit_one_cycle(2, slice(1e-3/ (2.6**4), 1e-3))